In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Lambda

# Helper libraries
import numpy as np

Using TensorFlow backend.


In [2]:
data_dimension = 128

X1 = np.load(r'C:\Users\Asus\Downloads\data\processed_data\x1.npy')
X2 = np.load(r'C:\Users\Asus\Downloads\data\processed_data\x2.npy')
Y = np.load(r'C:\Users\Asus\Downloads\data\processed_data\y.npy')

X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

(36, 49152)
(36, 49152)
(36,)


In [3]:
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

In [6]:
# Keras Sequential Model
tf.random.set_seed(100)

tf.keras.backend.clear_session()
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.3)


dimen = 128
input_shape = ((dimen ** 2) * 3,)
convolution_shape = (128 , 128 , 3 )
kernel_size_1 = (4,4)
kernel_size_2 = (3,3)
pool_size_1 = (3,3)
pool_size_2 = (2,2)
strides = 1

model = None
seq_model = keras.Sequential([
    keras.layers.Reshape(input_shape=input_shape, target_shape = (128 , 128 , 3 )),
    keras.layers.Conv2D( 32, kernel_size=kernel_size_1 , strides=strides , activation=leaky_relu ),
    keras.layers.Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation=leaky_relu),
    keras.layers.MaxPooling2D(pool_size=pool_size_1, strides=strides),
    keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=leaky_relu ),
    keras.layers.Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation=leaky_relu ),
    keras.layers.MaxPooling2D(pool_size=pool_size_2 , strides=strides),
    keras.layers.Flatten(),
    keras.layers.Dense(64)
    ])
    

# seq_model = tf.keras.Sequential(seq_conv_model)

input_x1 = Input(shape=input_shape)
input_x2 = Input(shape=input_shape)

output_x1 = seq_model(X1)
output_x2 = seq_model(X2)

distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
model = models.Model( [ input_x1 , input_x2 ] , outputs )

model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001))

initial_time = time.time()
model.fit(X ,Y ,
            batch_size=hyperparameters[ 'batch_size' ] ,
            epochs=hyperparameters[ 'epochs' ] ,
            callbacks=hyperparameters[ 'callbacks'],
            validation_data=hyperparameters[ 'val_data' ])

final_time = time.time()
eta = ( final_time - initial_time )
time_unit = 'seconds'
if eta >= 60 :
    eta = eta / 60
    time_unit = 'minutes'
model.summary()
print('Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit))
model.save(r'C:\Users\Asus\Downloads\data\model.h5')

ResourceExhaustedError: OOM when allocating tensor with shape[846400,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: dense/kernel/Initializer/random_uniform/

In [ ]:
# from keras.layers import Input

In [43]:
import numpy as np
import cv2
from PIL import Image

from keras.models import Model
from keras import layers
from keras.layers import Activation, Lambda
from keras.layers import Dense, concatenate, Flatten
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D, ZeroPadding2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras_applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.preprocessing import image
from mtcnn.mtcnn import MTCNN


!git clone https://github.com/TessFerrandez/research-papers.git
%cd research-papers/facenet
import utils
from utils import LRN2D
from model import create_model
import glob
import os
%cd C:/Users/Asus/Downloads/

C:\Users\Asus\Downloads\research-papers\facenet
C:\Users\Asus\Downloads


fatal: destination path 'research-papers' already exists and is not an empty directory.


In [2]:
myInput = Input(shape=(96, 96, 3))

x = ZeroPadding2D(padding=(3, 3), input_shape=(96, 96, 3))(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)
x = Lambda(LRN2D, name='lrn_1')(x)
x = Conv2D(64, (1, 1), name='conv2')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = Conv2D(192, (3, 3), name='conv3')(x)
x = BatchNormalization(axis=3, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = Lambda(LRN2D, name='lrn_2')(x)
x = ZeroPadding2D(padding=(1, 1))(x)
x = MaxPooling2D(pool_size=3, strides=2)(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2)(x)
inception_3a_pool = Conv2D(32, (1, 1), name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)))(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=3)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1))(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2))(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

inception_3b_pool = Lambda(lambda x: x**2, name='power2_3b')(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: x*9, name='mult9_3b')(inception_3b_pool)
inception_3b_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_3b')(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4))(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=3, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=3)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2)(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=3)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))

inception_4a_pool = Lambda(lambda x: x**2, name='power2_4a')(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: x*9, name='mult9_4a')(inception_4a_pool)
inception_4a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_4a')(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=3)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2)(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)))(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=3)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))

inception_5a_pool = Lambda(lambda x: x**2, name='power2_5a')(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3))(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: x*9, name='mult9_5a')(inception_5a_pool)
inception_5a_pool = Lambda(lambda x: K.sqrt(x), name='sqrt_5a')(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=3)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2)(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1))(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=3)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(dense_layer)


# Final Model
model = Model(inputs=[myInput], outputs=norm_layer)

In [3]:
model.load_weights('C:/Users/Asus/Downloads/research-papers/facenet/weights/nn4.small2.v1.h5')

In [31]:
def image_to_embedding(image, model):
#     image = cv2.imread(image)
#     image = Image.fromarray(image)
    image = np.resize(image,(96,96,3))  
    img = image[...,::-1]
    img = np.around(np.transpose(img, (0,1,2))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

In [5]:
img_embed_1 = image_to_embedding('WIN_20200608_19_41_39_Pro.jpg',model)

In [6]:
def recognize_face(face_image, input_embeddings, model):

    embedding = image_to_embedding(face_image, model)
    
    minimum_distance = 200
    name = 'Shady'
    
    # Loop over  names and encodings.
    for (input_name, input_embedding) in input_embeddings.items():
        
       
        euclidean_distance = np.linalg.norm(embedding-input_embedding)
        

        print('Euclidean distance from %s is %s' %(input_name, euclidean_distance))

        
        if euclidean_distance < minimum_distance:
            minimum_distance = euclidean_distance
            name = input_name
    
    if minimum_distance < 0.68:
        return name
    else:
        return 'unknown'

In [51]:
# in_encoder = Normalizer(norm='l2')
# model = pickle.load(open(filename, 'rb'))

cap = cv2.VideoCapture(0)
blank_image = np.zeros((224,224,3), np.uint8)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

while(True):
    ret, frame = cap.read()
    detector = MTCNN()
    results = detector.detect_faces(frame)
    if len(results) > 0:
        for result in results:
            try:
                x1, y1, width, height = result['box']
                x2, y2 = x1 + width, y1 + height
                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),3)
                face = frame[y1:y2,x1:x2]
                face_embed = image_to_embedding(face, model)
                name = recognize_face(face_embed, input_embeddings, model)
                frame = cv2.putText(img, str(name), (x1+5,y1-5), font, 1, (255,255,255), 2)
                cv2.imshow('frame',frame)
            except:
#                 frame = cv2.putText(frame, 'No Person', (10,10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
                pass
                
    else:
        frame = cv2.putText(frame, 'No Person', (10,10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
        cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
# out.release()
cv2.destroyAllWindows()

Euclidean distance from Shady is 0.6568576
Euclidean distance from unkown is 1.1366795
Euclidean distance from Shady is 0.65686244
Euclidean distance from unkown is 1.1366293
Euclidean distance from Shady is 0.6568277
Euclidean distance from unkown is 1.1367179
Euclidean distance from Shady is 0.6567487
Euclidean distance from unkown is 1.1366817
Euclidean distance from Shady is 0.6568827
Euclidean distance from unkown is 1.1366888
Euclidean distance from Shady is 0.6568917
Euclidean distance from unkown is 1.1367035
Euclidean distance from Shady is 0.65701115
Euclidean distance from unkown is 1.1367918
Euclidean distance from Shady is 0.65691155
Euclidean distance from unkown is 1.1367251
Euclidean distance from Shady is 0.6569022
Euclidean distance from unkown is 1.1366662
Euclidean distance from Shady is 0.65654075
Euclidean distance from unkown is 1.1365327
Euclidean distance from Shady is 0.65695477
Euclidean distance from unkown is 1.1367357
Euclidean distance from Shady is 0.656

In [8]:
cap.release()
cv2.destroyAllWindows()

In [50]:
image1 = r'C:\Users\Asus\Downloads\WIN_20200608_19_41_39_Pro.jpg'
image1 = cv2.imread(image1)

detector = MTCNN()
results = detector.detect_faces(image1)
x1, y1, width, height = result['box']
x2, y2 = x1 + width, y1 + height
cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),3)
face = frame[y1:y2,x1:x2]
# image = Image.fromarray(face)

face_embed = image_to_embedding(face, model)
name = recognize_face(face_embed, input_embeddings, model)
print(name)

Euclidean distance from Shady is 0.6566331
Euclidean distance from unkown is 1.1365725
Shady


In [25]:
# face.shape


(291, 240, 3)

In [30]:
# face = np.resize(image,(96,96,3))
# face.shape

(96, 96, 3)

In [35]:
def create_input_image_embeddings():
    input_embeddings = {}

    for file in glob.glob("images/*.jpg"):
        person_name = os.path.splitext(os.path.basename(file))[0]
        image_file = cv2.imread(file, 1)
        input_embeddings[person_name] = image_to_embedding(image_file, model)

    return input_embeddings

In [48]:
input_embeddings = create_input_image_embeddings()

In [49]:
input_embeddings

{'Shady': array([[ 0.11269069, -0.02725138,  0.02528271, -0.03003384,  0.19955336,
          0.15502441,  0.11091311,  0.09558969, -0.12599544, -0.09113885,
         -0.04732256,  0.03435574,  0.03417895, -0.01835501,  0.09427758,
         -0.12442696,  0.03826237, -0.00762649, -0.05759399,  0.17291562,
          0.03857092, -0.00780049,  0.14733547, -0.01718669, -0.02101509,
         -0.09907557, -0.11743291, -0.11763415,  0.05853102, -0.10355877,
          0.00397422, -0.0358151 , -0.06605572,  0.04779268,  0.03045714,
         -0.01761821,  0.01693694,  0.20529778, -0.09206975, -0.10801191,
          0.12844951,  0.08469838, -0.14445613, -0.03563142, -0.31722674,
          0.05839279, -0.01355324,  0.0032838 , -0.07823554,  0.07343209,
         -0.12819508, -0.02275436, -0.03083704,  0.04168827,  0.02543081,
         -0.08415215,  0.03544803,  0.11925613, -0.11001533, -0.05613751,
         -0.06237664,  0.06141718,  0.11285627, -0.17225137,  0.16134135,
         -0.00398265,  0.0327

In [10]:
def prepare_images_from_dir(dir_path , flatten=True) :
    images = list()
    images_names = os.listdir( dir_path )
    for imageName in images_names :
        image = Image.open(dir_path + imageName)
        resize_image = image.resize((128, 128))
        array = list()
        for x in range(128):
            sub_array = list()
        for y in range(128):
            sub_array.append(resize_image.load()[x, y])
        array.append(sub_array)
        image_data = np.array(array)
        image = np.array(np.reshape(image_data,(128, 128, 3)))
        images.append(image)
            
    if flatten :
        images = np.array(images)
        return images.reshape( ( images.shape[0]  , 128**2 * 3  ) ).astype( np.float32 )
    else:
        return np.array(images)